<a href="https://colab.research.google.com/github/mrvoral/Comp541-Project/blob/master/Lab4_Merve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Julia on Colaboratory ##

[Colaboratory](https://colab.research.google.com) does not provide native support for the [Julia programming language](https://julialang.org). However, since Colaboratory gives you root access to the machine that runs your notebook (the *“runtime”* in Colaboratory terminology), we can install Julia support by uploading a specially crafted Julia notebook  – *this* notebook. We then install Julia and [IJulia](https://github.com/JuliaLang/IJulia.jl) ([Jupyter](https://jupyter.org)/Colaboratory notebook support) and reload the notebook so that Colaboratory detects and initiates what we installed.

In brief:

1. **Run the cell below**
2. **Reload the page**
3. **Edit the notebook name and start hacking Julia code below**

**If your runtime resets**, either manually or if left idle for some time, **repeat steps 1 and 2**.

### Acknowledgements ###

This hack by Pontus Stenetorp is an adaptation of [James Bradbury’s original Colaboratory Julia hack](https://discourse.julialang.org/t/julia-on-google-colab-free-gpu-accelerated-shareable-notebooks/15319/27), that broke some time in September 2019 as Colaboratory increased their level of notebook runtime isolation. There also appears to be CUDA compilation support installed by default for each notebook runtime type in October 2019, which shaves off a good 15 minutes or so from the original hack’s installation time.

In [0]:
# Installation cell
%%shell
if ! command -v julia 2>&1 > /dev/null
then
    wget 'https://julialang-s3.julialang.org/bin/linux/x64/1.3/julia-1.3.1-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
fi
julia -e 'using Pkg; pkg"add Plots; add PyPlot; add IJulia; add Knet;"'
julia -e 'using Pkg; pkg"build Knet;"'

In [3]:
using Knet: Knet, conv4, pool, mat, KnetArray, nll, zeroone, progress, sgd, param, param0, dropout, relu, Data, gpu, softmax
Knet.gpu()

┌ Warning: You are using CUDNN 7.6.5 for CUDA 10.1.0 with CUDA toolkit 10.0.145; these might be incompatible.
└ @ CuArrays /root/.julia/packages/CuArrays/1njKF/src/CuArrays.jl:122


0

In [0]:
# Set display width, load packages, import symbols
ENV["COLUMNS"]=72
using Pkg; for p in ("Knet","IterTools"); haskey(Pkg.installed(),p) || Pkg.add(p); end
using Knet, IterTools

In [0]:
# Define convolutional layer:
struct Conv; w; b; f; end
(c::Conv)(x) = pool(c.f.(conv4(c.w, x;padding=0) .+ c.b),window=(2,2),stride=2)
Conv(w1,w2,cx,cy,f=relu) = Conv(param(w1,w2,cx,cy), param0(1,1,cy,1), f);

In [0]:
# Define dense layer:
struct Dense; w; b; f; end
(d::Dense)(x) = d.f.(d.w * mat(x) .+ d.b)
Dense(i::Int,o::Int,f=sigm) = Dense(param(o,i), param0(o), f);

In [0]:
# Define a chain of layers:
struct Chain; layers; Chain(args...)=new(args); end
(c::Chain)(x) = (for l in c.layers; x = l(x); end; x)
(c::Chain)(x,y) = nll(c(x),y)

In [9]:
# Load MNIST data
include(Knet.dir("data","mnist.jl"))
dtrn, dtst = mnistdata();

┌ Info: Loading MNIST...
└ @ Main /root/.julia/packages/Knet/2xiR8/data/mnist.jl:33


In [49]:
# Train and test LeNet (about 30 secs on a gpu to reach 99% accuracy)
LeNet = Chain(Conv(5,5,1,3), Dense(12*12*3,10))
progress!(adam(LeNet, ncycle(dtrn,100)))
accuracy(LeNet, dtst)

┣████████████████████┫ [100.00%, 60000/60000, 01:55/01:55, 520.93i/s] 


0.9876

In [0]:
# Train and test LeNet (about 30 secs on a gpu to reach 99% accuracy)
D1=10
LeNet = Chain(Conv(2,2,1,10), Conv(3,3,10,50),Conv(5,5,50,D1),Dense(D1,10))
progress!(adam(LeNet, ncycle(dtrn,100)))
1-accuracy(LeNet,dtrn)

┣███████████▌        ┫ [57.52%, 34513/60000, 01:53/03:17, 312.34i/s] 